# 参数管理

我们首先关注具有单隐藏层的多层感知机

In [1]:
import mindspore
import numpy as np
from mindspore import nn
from mindspore import Tensor

net = nn.SequentialCell([nn.Dense(4, 8), nn.ReLU(), nn.Dense(8, 1)])
X = Tensor(np.random.rand(2, 4), mindspore.float32)
net(X)

Tensor(shape=[2, 1], dtype=Float32, value=
[[-8.20011774e-05],
 [-1.58124196e-04]])

参数访问

In [3]:
print(net[2].parameters_dict())

OrderedDict([('2.weight', Parameter (name=2.weight, shape=(1, 8), dtype=Float32, requires_grad=True)), ('2.bias', Parameter (name=2.bias, shape=(1,), dtype=Float32, requires_grad=True))])


目标参数

In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'mindspore.common.parameter.ParameterTensor'>
Parameter (name=2.bias, shape=(1,), dtype=Float32, requires_grad=True)
Parameter (name=2.bias, shape=(1,), dtype=Float32, requires_grad=True)


一次性访问所有参数

In [16]:
print(*[(name, param.shape) for name, param in net[0].parameters_dict().items()])
print(*[(name, param.shape) for name, param in net.parameters_dict().items()])

('0.weight', (8, 4)) ('0.bias', (8,))
('0.weight', (8, 4)) ('0.bias', (8,)) ('2.weight', (1, 8)) ('2.bias', (1,))


In [19]:
net.parameters_dict()['2.bias'].data

Parameter (name=2.bias, shape=(1,), dtype=Float32, requires_grad=True)

从嵌套块收集参数

In [20]:
def block1():
    return nn.SequentialCell([nn.Dense(4, 8), nn.ReLU(),
                              nn.Dense(8, 4), nn.ReLU()])

def block2():
    net = nn.SequentialCell()
    for i in range(4):
        net.append(block1())
    return net

rgnet = nn.SequentialCell([block2(), nn.Dense(4, 1)])
rgnet(X)

Tensor(shape=[2, 1], dtype=Float32, value=
[[ 5.73089381e-18],
 [ 4.57987654e-18]])

我们已经设计了网络，让我们看看它是如何组织的

In [21]:
print(rgnet)

SequentialCell<
  (0): SequentialCell<
    (0): SequentialCell<
      (0): Dense<input_channels=4, output_channels=8, has_bias=True>
      (1): ReLU<>
      (2): Dense<input_channels=8, output_channels=4, has_bias=True>
      (3): ReLU<>
      >
    (1): SequentialCell<
      (0): Dense<input_channels=4, output_channels=8, has_bias=True>
      (1): ReLU<>
      (2): Dense<input_channels=8, output_channels=4, has_bias=True>
      (3): ReLU<>
      >
    (2): SequentialCell<
      (0): Dense<input_channels=4, output_channels=8, has_bias=True>
      (1): ReLU<>
      (2): Dense<input_channels=8, output_channels=4, has_bias=True>
      (3): ReLU<>
      >
    (3): SequentialCell<
      (0): Dense<input_channels=4, output_channels=8, has_bias=True>
      (1): ReLU<>
      (2): Dense<input_channels=8, output_channels=4, has_bias=True>
      (3): ReLU<>
      >
    >
  (1): Dense<input_channels=4, output_channels=1, has_bias=True>
  >


In [24]:
rgnet[0][1][0].bias.data

Parameter (name=0.1.0.bias, shape=(8,), dtype=Float32, requires_grad=True)

内置初始化

In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0017,  0.0145,  0.0104, -0.0013]), tensor(0.))

In [11]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对某些块应用不同的初始化方法

In [12]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.6136, -0.1642, -0.2092,  0.5903])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, 7.5857, 0.0000, -0.0000],
        [0.0000, 0.0000, 0.0000, -0.0000]], grad_fn=<SliceBackward>)

In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  8.5857,  1.0000,  1.0000])

参数绑定

In [15]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
